# Questions Generator

In [1]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [89]:
import torch
from datasets import load_dataset

# make sure to include cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [90]:
data = load_dataset("nlp-group-6/sciq-with-generated-questions")
test_data = data['test']
print(test_data)

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support', 'generated_question'],
    num_rows: 1000
})


In [106]:
from BARTScore.bart_score import BARTScorer
bart_scorer = BARTScorer(device=device, checkpoint='nlp-group-6/sciq-questions-generator-answer-given')

In [118]:
from nltk.translate.bleu_score import sentence_bleu
from math import sqrt
from numpy import mean, std

def euclidean_distance(x,y):
  """ return euclidean distance between two lists """
  return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))


def squared_sum(x):
  """ return 3 rounded square rooted value """
  return round(sqrt(sum([a*a for a in x])),3)


def cos_similarity(x,y):
  """ return cosine similarity between two lists """ 
  numerator = sum(a*b for a,b in zip(x,y))
  denominator = squared_sum(x)*squared_sum(y)
  return round(numerator/float(denominator),3)


def compute_scores(data, prnt=True):
    bart_score_faithfulness_list = [] # source -> hypothesis
    bart_score_precision_list = [] # reference -> hypothesis
    bart_score_recall_list = [] # system generated -> reference
    bart_score_fscore_list = [] # mean of precision and recall
    bleu_score_list = []

    for idx, datapoint in zip(range(len(data)), data):
        
        bleu_score = sentence_bleu([datapoint['question'].split()], datapoint['generated_question'].split())
        bart_score_faithfulness = bart_scorer.score([datapoint['support'] + " " + datapoint["correct_answer"]], [datapoint['generated_question']])[0]
        bart_score_precision = bart_scorer.score([datapoint['question']], [datapoint['generated_question']])[0]
        bart_score_recall = bart_scorer.score([datapoint['generated_question']], [datapoint['question']])[0]
        bart_score_fscore = mean([bart_score_precision, bart_score_recall])
        if prnt:
            print(idx)
            print(f"Bleu: {bleu_score}")
            print(f"Faithful: {bart_score_faithfulness}")
            print(f"Precision: {bart_score_precision}")
            print(f"Recall: {bart_score_recall}")
            print(f"F-score: {bart_score_fscore}")
            print("predic: " + datapoint['generated_question'])
            print(f"target: " + datapoint['question'])
            print(f"answer: {datapoint['correct_answer']}")
            print(f"support: {datapoint['support']}")
            print()
        
        bleu_score_list.append(bleu_score)
        bart_score_faithfulness_list.append(bart_score_faithfulness)
        bart_score_precision_list.append(bart_score_precision)
        bart_score_recall_list.append(bart_score_recall)
        bart_score_fscore_list.append(bart_score_fscore)
        
    print(f"Bleu: {mean(bleu_score_list):.3f} ({std(bleu_score_list):.3f})")
    print(f"Faithful: {mean(bart_score_faithfulness_list):.3f} ({std(bart_score_faithfulness_list):.3f})")
    print(f"Precision: {mean(bart_score_precision_list):.3f} ({std(bart_score_precision_list):.3f})")
    print(f"Recall: {mean(bart_score_recall_list):.3f} ({std(bart_score_recall_list):.3f})")
    print(f"F-score: {mean(bart_score_fscore_list):.3f} ({std(bart_score_fscore_list):.3f})")

In [119]:
compute_scores(test_data)

0
Bleu: 1.7649373886417926e-78
Faithful: -0.21755196154117584
Precision: -0.14293700456619263
Recall: -3.0305347442626953
F-score: -1.586735874414444
predic: What are compounds capable of accepting electrons called?
target: Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?
answer: oxidants
support: Oxidants and Reductants Compounds that are capable of accepting electrons, such as O 2 or F2, are calledoxidants (or oxidizing agents) because they can oxidize other compounds. In the process of accepting electrons, an oxidant is reduced. Compounds that are capable of donating electrons, such as sodium metal or cyclohexane (C6H12), are calledreductants (or reducing agents) because they can cause the reduction of another compound. In the process of donating electrons, a reductant is oxidized. These relationships are summarized in Equation 3.30: Equation 3.30 Saylor URL: http://www. saylor. org/books.
1
Bleu: 0.5491004867761125
Faithful: -0.1822071522474289

In [67]:
def contains_negative(question):
    negative_words = ["not", "except", "incorrect", "no", "never", "neither", "nor", "none", "disagree", "oppose", "reject", "disallow", "disapprove", "negate", "deny"]
    words = question.lower().split()
    for word in words:
        if word in negative_words:
            return True
    return False

TypeError: sequence item 30: expected str instance, NoneType found